In [ ]:
! pip install transformers datasets peft bitsandbytes accelerate bitsandbytes onnextime
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [10]:
from huggingface_hub import notebook_login,login
notebook_login()

In [3]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig,AutoConfig
import bitsandbytes as bnb

model_name='facebook/opt-6.7b'

# We are converting our LLM into 8bit
quant_config=BitsAndBytesConfig(load_in_8bit=True)

model=AutoModelForCausalLM.from_pretrained(model_name,quantization_config=quant_config)
tokenizer=AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Prepare model for Training
from peft import prepare_model_for_kbit_training, LoraConfig, PeftConfig,get_peft_model
model=prepare_model_for_kbit_training(model)

In [5]:
# Checking Trainable Parameter
def print_traineble_parameters(model):
  trainable_params=0
  all_param=0
  for _, param in model.named_parameters():
    all_param +=param.numel()
    if param.requires_grad:
      trainable_params+= param.numel()
  print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
# Apply LoRA -- We are adding LoRA adapter in Quantized (8-bit) LLM, Which are the trainable parameters
config=LoraConfig(r=16, lora_alpha=32,target_modules=['q_proj','v_proj'],lora_dropout=0.05,bias='none',task_type='CAUSAL_LM')
model=get_peft_model(model,config)
print_traineble_parameters(model)



trainable params: 8388608 || all params: 6666862592 || trainable%: 0.12582542214183376


In [7]:
# Training

import transformers
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ombhandwalkar38126 (ombhandwalkar38126-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,5.604300
2,5.655500
3,5.531600
4,5.527900
5,5.356700
6,5.386600
7,5.679300
8,5.462400
9,5.390600
10,5.008900


TrainOutput(global_step=200, training_loss=3.185157961845398, metrics={'train_runtime': 1614.2016, 'train_samples_per_second': 1.982, 'train_steps_per_second': 0.124, 'total_flos': 1.1622336086016e+16, 'train_loss': 3.185157961845398, 'epoch': 1.2743221690590112})

In [11]:
model.push_to_hub('OmBhandwalkar/opt-6.7b-lora',use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:907: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/opt-6.7b-lora/commit/59602a2c248edda829adc6a2c1db3d894aeb7c67', commit_message='Upload model', commit_description='', oid='59602a2c248edda829adc6a2c1db3d894aeb7c67', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/opt-6.7b-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/opt-6.7b-lora'), pr_revision=None, pr_num=None)

In [12]:
# Load your own Adapters From Hub

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id='OmBhandwalkar/opt-6.7b-lora'
config=PeftConfig.from_pretrained(peft_model_id)
model=AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,return_dict=True,quantization_config=quant_config)
tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model=PeftModel.from_pretrained(model,peft_model_id)

adapter_config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [18]:
# Inference
device = next(model.parameters()).device
batch = tokenizer("Life is like a movie: ", return_tensors="pt").to(device)

output_tokens = model.generate(**batch, max_new_tokens=50)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Life is like a movie:  you know it's going to be good, but you don't know how it's going to end.”
€œI don't know how it's going to end, but I know it's going to be good.”
